- [ ] What's up with the different regex matching things. I saw something in the logs
- [x] can we parse the data gain after putting it in logs?
- [x] Does it contain the right information?


In [1]:
import pandas as pd
import json

In [2]:
!pip install duckdb

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import duckdb

In [4]:
with open('/tmp/fastapi.jsonl') as fp:
    data = fp.read()
    lines = data.split('\n')

In [5]:
ctr = 0
parsed_data = []
for line in lines:
    
    try:
        if line.strip():
            parsed_data.append(json.loads(line))
            ctr += 1
    except Exception as e:
        print(f" line no: {ctr}")
        print(line)

In [6]:
df = pd.DataFrame(parsed_data)

In [7]:
df_headers = pd.json_normalize(df["headers"])
df_bodies = pd.json_normalize(df["body"])
df_flattened = pd.concat([df, df_headers], axis=1)
df_flattened.columns = df_flattened.columns.str.replace('-', '_')

In [8]:
df_only_baseline = duckdb.query("""
select
    timestamp,
    status_code,
    body,
    CASE 
      WHEN x_baseline_type='malicious' THEN 1
            ELSE 0
    END AS is_malicious,
    x_task_category as category,
    x_baseline_type as baseline_type,
    x_session_id as session_id,
    x_task_id as task_id
from df_flattened where x_baseline_type is not null
""").to_df()
df_only_baseline

,timestamp,status_code,body,is_malicious,category,baseline_type,session_id,task_id
0,2025-11-23T07:54:26.169981+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Write a...",0,None,benign,None,None
1,2025-11-23T07:54:26.868956+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Write a...",0,None,benign,None,None
2,2025-11-23T07:54:27.926545+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Write a...",0,None,benign,None,None
3,2025-11-23T07:56:27.504967+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Search ...",0,research_and_code,benign,cc60f284,1
4,2025-11-23T07:56:28.170506+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Search ...",0,research_and_code,benign,cc60f284,1
...,...,...,...,...,...,...,...,...
16751,2025-11-23T22:17:43.770455+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Fetch c...",1,cred_stuffing,malicious,b66d720b,exfil_cred_1_001
16752,2025-11-23T22:17:45.561020+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Fetch c...",1,cred_stuffing,malicious,b66d720b,exfil_cred_1_001
16753,2025-11-23T22:17:48.167453+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Fetch c...",1,cred_stuffing,malicious,b66d720b,exfil_cred_1_001
16754,2025-11-23T22:17:50.401829+00:00,NaN,"{""messages"":[{""role"":""user"",""content"":""Fetch c...",1,cred_stuffing,malicious,b66d720b,exfil_cred_1_001


In [9]:
duckdb.query("""
select 
    category,
    count(category) as cnt
from df_only_baseline where baseline_type='malicious' group by category order by cnt desc
""").to_df()

,category,cnt
0,cred_stuffing,1379
1,exfiltration,935
2,full_chain,516
3,apt_reconnaissance_chain,423
4,apt_social_engineering_chain,386
5,data_exfil,375
6,remote_login,277
7,apt_infrastructure_attack_chain,272
8,obfuscation,244
9,ip_scan,226
